In [16]:
alpha_name = "AlphaNetV1_Shift_5_Sequence_6_1d_ret_standscalar"
start_date = '2019-01-01'
end_date = '2021-06-01'
change = 5

In [17]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
import numpy as np
import pandas as pd
import DataAPI
import os
input_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/' + alpha_name
output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/'

alpha_list = []
trading_list = DataAPI.get_trading_days(start_date, end_date)
count = 0
for day in trading_list:
    file = os.path.join(input_path, str(day.year), day.strftime('%Y%m%d') + '.csv')
    try:
        df = pd.read_csv(file, dtype={0: str})
    except FileNotFoundError:
        count += 1
        continue
    flag = count % change
    path_output = os.path.join(output_path, str(alpha_name + "_" + str(flag)), str(day.year), day.strftime('%Y%m%d') + '.csv')
    alpha_list.append(str(alpha_name + "_" + str(flag)))
    if not os.path.exists(path_output):
        os.makedirs(os.path.dirname(path_output), exist_ok=True)
    df.to_csv(path_output, index=False, header=False)
    count += 1

In [18]:
pd.set_option('expand_frame_repr', False)
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib")
alpha_list = list(set(alpha_list))
print(alpha_list)
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)
FT.load_return_data()

writing_path = r'/home/wuwenjun/Desktop/%s.txt' % alpha_name
f = open (writing_path,'w')
print("\n",file=f)
f.close()
for alpha_name in alpha_list:
    FT.get_intersection_ic(feature_name=alpha_name, truncate_fold=None, method='spearman',
                                     period=('1d', '3d', '5d'))
    ic_flag, trading_direction = FT.test_ic(alpha_name, verbose=False)
    df, df_all = FT.get_ic_summary_by_month(num=6)
    
    if trading_direction == -1:
        negative = True
    else:
        negative = False
    f = open (writing_path,'a')
    print("*"*100,end="\n"*3,file=f)
    print(alpha_name,end = "\n"*3,file=f)
    print(df,end="\n"*3,file = f)
    a = FT.get_top_return(alpha_name, negative= False, trade_type='long-only', transmission_rate=0.00025,
                            tax_rate=0.001, verbose=True,bt_price = "vwap",trade_period=6)
    print(a[1],end = "\n"*3+"*"*100 ,file = f)
    f.close()

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:   5%|████                                                                                     | 133/2920 [00:00<00:03, 765.44it/s]

['AlphaNetV1_Shift_5_Sequence_6_1d_ret_standscalar_1', 'AlphaNetV1_Shift_5_Sequence_6_1d_ret_standscalar_0', 'AlphaNetV1_Shift_5_Sequence_6_1d_ret_standscalar_4', 'AlphaNetV1_Shift_5_Sequence_6_1d_ret_standscalar_2', 'AlphaNetV1_Shift_5_Sequence_6_1d_ret_standscalar_3']


Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 584/584 [00:01<00:00, 506.75it/s]



IC table for feature "AlphaNetV1_Shift_5_Sequence_6_1d_ret_standscalar_1" (mean std IR):
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      period       |  IC_AlphaNetV1_1d  |  IC_AlphaNetV1_2d  |  IC_AlphaNetV1_3d  |  IC_AlphaNetV1_4d  |  IC_AlphaNetV1_5d  |
+===================+====================+====================+====================+====================+====================+
| 20190103-20190630 |   2.76 3.66 0.75   |   3.15 4.39 0.72   |   3.57 4.29 0.83   |   4.05 4.25 0.95   |   4.10 4.64 0.88   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20190701-20191231 |   2.89 3.85 0.75   |   3.52 3.81 0.93   |   3.50 3.62 0.97   |   3.42 3.81 0.90   |   3.43 3.46 0.99   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20200101-20200630 |

...


AlphaNetV1_Shift_5_Sequence_6_1d_ret_standscalar_2 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+--------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period       |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+==========================+===================+============+
| 20190102-20190630 |     210     |   1.48   |   8.14    |    3.05    |   0.39   |    58.97    | 5.33(20190307-20190506)  |  2.00(20190429)   |   16.41    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+--------------------------+-------------------+------------+
| 20190701-20191231 |     214     |  -3.95   |   6.08    |   -7.42    |  -1.34   |    50.

Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 584/584 [00:01<00:00, 376.45it/s]



IC table for feature "AlphaNetV1_Shift_5_Sequence_6_1d_ret_standscalar_3" (mean std IR):
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      period       |  IC_AlphaNetV1_1d  |  IC_AlphaNetV1_2d  |  IC_AlphaNetV1_3d  |  IC_AlphaNetV1_4d  |  IC_AlphaNetV1_5d  |
+===================+====================+====================+====================+====================+====================+
| 20190107-20190630 |   4.10 4.49 0.91   |   4.79 3.66 1.31   |   4.66 4.03 1.16   |   4.55 3.93 1.16   |   4.86 3.79 1.28   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20190701-20191231 |   3.66 4.09 0.90   |   3.59 3.80 0.95   |   4.21 4.16 1.01   |   4.28 4.12 1.04   |   4.39 4.33 1.01   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20200101-20200630 |

Getting BT result for date: 20210531...: 100%|██████████████████████████████████████████████████████████████████████| 584/584 [00:06<00:00, 90.42it/s]



AlphaNetV1_Shift_5_Sequence_6_1d_ret_standscalar_3 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+--------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period       |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+==========================+===================+============+
| 20190102-20190630 |     210     |  -0.23   |   7.38    |   -0.46    |  -0.07   |    51.28    | 5.65(20190109-20190131)  |  1.52(20190129)   |   15.67    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+--------------------------+-------------------+------------+
| 20190701-20191231 |     214     |  -1.54   |   5.46    |   -2.92    |  -0.57   |    48.